In [123]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
import seaborn as sns
import matplotlib.pyplot as plt
from time import time
from sklearn.manifold import TSNE
from scipy import stats


In [116]:
real = pd.read_csv("complete_data_except_age_at_death.csv")
synthetic = pd.read_csv("autoencoded_complete_data_except_age_at_death.csv")
print(real.shape, synthetic.shape)

(6564, 3084) (6564, 3077)


In [117]:
real = real.drop(columns=['icustay_id','hadm_id','subject_id','first_wardid',
                          'last_wardid','has_chartevents_data','ser_ind'])

In [120]:
icdcols = [c for c in real.columns.tolist() if 'p_icd' in c]


columns=['hospital_expire_flag'
         #,'expire_flag'
         #,'los_total'
         #,'died'
         ,'age_at_death'
        ]
columns.extend(icdcols)
Xr = real.drop(columns=columns).values
Xs = synthetic.drop(columns=columns).values




In [121]:
t= time()
Xr_embed = TSNE(n_components=3).fit_transform(Xr)
Xs_embed = TSNE(n_components=3).fit_transform(Xs)
print("took {}".format(time()-t))

took 1212.6495056152344


In [93]:
Yr = real[['hospital_expire_flag']].values
Ys = synthetic[['hospital_expire_flag']].values

TSNErl = np.concatenate((Xr_embed,Yr),axis=1)
TSNEsl = np.concatenate((Xs_embed,Ys),axis=1)

In [94]:
TSNErall = pd.DataFrame(TSNErl, columns=["x","y","z","died"])
TSNEsall = pd.DataFrame(TSNEsl, columns=["x","y","z","died"])

In [95]:
TSNE_ralive = TSNErall[TSNErall["died"]==0]
TSNE_rdied = TSNErall[TSNErall["died"]==1]
TSNE_salive = TSNEsall[TSNEsall["died"]==0]
TSNE_sdied = TSNEsall[TSNEsall["died"]==1]

In [96]:
TSNE_ralive = TSNE_ralive.drop(columns='died')
TSNE_rdied = TSNE_rdied.drop(columns='died')
TSNE_salive = TSNE_salive.drop(columns='died')
TSNE_sdied = TSNE_sdied.drop(columns='died')

In [97]:
TSNE_ralive.to_csv('/home/ubuntu/analyze_shallow/TSNE_r_alive.csv')
TSNE_rdied.to_csv('/home/ubuntu/analyze_shallow/TSNE_r_died.csv')
TSNE_salive.to_csv('/home/ubuntu/analyze_shallow/TSNE_s_alive.csv')
TSNE_sdied.to_csv('/home/ubuntu/analyze_shallow/TSNE_s_died.csv')

In [100]:
XrXr = real.drop(columns=columns)

In [124]:
t ,p = stats.ttest_ind(real['age_at_death'].dropna(), synthetic['age_at_death'].dropna())

In [149]:
def get_pval(v1):
    t ,p = stats.ttest_ind(real[v1].dropna(), synthetic[v1].dropna())
    return '''ttest result:   p = %g  
    t = %g '''  %(p,t)

In [150]:
get_pval('avrg_16_1')

'ttest result:   p = 5.25442e-54  \n    t = 15.5444 '